In [119]:
import pandas as pd
import requests

key='fcd98255352c8ed6746f9be1c09b600069bc7aa1'

In [120]:
params={'request':'all','page':0}
url = "https://steamspy.com/api.php"
response = requests.get(url,params=params)

if response.status_code == 200:
    data = response.json()
    games_list = list(data.values())
    games = pd.DataFrame(games_list)
else:
    print("Failed to fetch data from SteamSpy.")
    games=pd.DataFrame()

games

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,730,Counter-Strike: Global Offensive,Valve,Valve,,7642084,1173003,0,"100,000,000 .. 200,000,000",32929,729,5842,331,0,0,0,1474820
1,1172470,Apex Legends,Respawn,Electronic Arts,,668053,326926,0,"100,000,000 .. 200,000,000",8560,561,651,234,0,0,0,203490
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",,1520457,1037487,0,"100,000,000 .. 200,000,000",22545,849,5598,366,0,0,0,608158
3,1623730,Palworld,Pocketpair,Pocketpair,,358266,22443,0,"50,000,000 .. 100,000,000",3413,404,2067,275,2999,2999,0,25658
4,440,Team Fortress 2,Valve,Valve,,1044264,117208,0,"50,000,000 .. 100,000,000",9250,465,349,140,0,0,0,43534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,490220,Prismata,Lunarch Studios,Lunarch Studios,,970,217,0,"1,000,000 .. 2,000,000",416,0,428,0,0,0,0,4
996,921060,Modern Combat 5,Gameloft,Gameloft,,3130,2595,0,"1,000,000 .. 2,000,000",195,0,29,0,0,0,0,0
997,274520,Darkwood,Acid Wizard Studio,Acid Wizard Studio,,20529,1089,0,"1,000,000 .. 2,000,000",249,0,91,0,1499,1499,0,76
998,70000,Dino D-Day,800 North and Digital Ranch,800 North and Digital Ranch,,12659,2391,0,"1,000,000 .. 2,000,000",201,0,150,0,999,999,0,3


If it's 999999, then data for this application is hidden on developer's request.

In [140]:
games=games[games['appid']!=999999]
games=games[['appid','name','developer','publisher','positive','negative','owners','initialprice']]
games['price']=games['initialprice']
games.drop(['initialprice'], axis=1, inplace=True)
games.reset_index(drop=True,inplace=True)
games['price']=pd.to_numeric(games['price'])/100
games.head()

,appid,name,developer,publisher,positive,negative,owners,id,date,price
0,730,Counter-Strike: Global Offensive,Valve,Valve,7642084,1173003,"100,000,000 .. 200,000,000",018d937f-7851-7004-b780-3f657a301f9a,2012-08-21,0.00
1,1172470,Apex Legends,Respawn,Electronic Arts,668053,326926,"100,000,000 .. 200,000,000",018d937f-0ddf-7119-9337-aa96d8a36d39,2019-02-04,0.00
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",1520457,1037487,"100,000,000 .. 200,000,000",018d937f-57ce-723f-b7e7-7c7b5df93471,2017-12-21,0.00
3,1623730,Palworld,Pocketpair,Pocketpair,358266,22443,"50,000,000 .. 100,000,000",018d937f-50c1-7086-807c-e020c98c72b2,2024-01-19,29.99
4,440,Team Fortress 2,Valve,Valve,1044264,117208,"50,000,000 .. 100,000,000",018d937e-fde4-72ff-a7af-45e4955a8dd6,2007-10-10,0.00


In [122]:
params={'key':key,'shop':'steam','country':'US'}

url=f'https://api.isthereanydeal.com/lookup/id/shop/61/v1'

response = requests.post(url,json=[f"app/{appid}" for appid in games['appid']],params=params)

games['id']=response.json().values()

In [123]:
params={'key':key}

url='https://api.isthereanydeal.com/games/info/v2'
date=[]

for idd in games['id']:
    params['id']=idd
    response = requests.get(url,params=params)
    if response.status_code != 200:
        continue
    data=response.json()
    date.append(data['releaseDate'])

games['date']=date

In [126]:
games['date']=pd.to_datetime(games['date'])

In [139]:
games.head()

,appid,name,developer,publisher,positive,negative,owners,id,date,price
0,730,Counter-Strike: Global Offensive,Valve,Valve,7642084,1173003,"100,000,000 .. 200,000,000",018d937f-7851-7004-b780-3f657a301f9a,2012-08-21,0.00
1,1172470,Apex Legends,Respawn,Electronic Arts,668053,326926,"100,000,000 .. 200,000,000",018d937f-0ddf-7119-9337-aa96d8a36d39,2019-02-04,0.00
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",1520457,1037487,"100,000,000 .. 200,000,000",018d937f-57ce-723f-b7e7-7c7b5df93471,2017-12-21,0.00
3,1623730,Palworld,Pocketpair,Pocketpair,358266,22443,"50,000,000 .. 100,000,000",018d937f-50c1-7086-807c-e020c98c72b2,2024-01-19,29.99
4,440,Team Fortress 2,Valve,Valve,1044264,117208,"50,000,000 .. 100,000,000",018d937e-fde4-72ff-a7af-45e4955a8dd6,2007-10-10,0.00


In [ ]:
params={'key':key,'country':'US','shops':61}
url='https://api.isthereanydeal.com/games/history/v2'
df=pd.DataFrame(columns=games.columns)

for idd in games['id']:
    current=games[games['id']==idd].to_dict()
    params['id']=idd
    params['since']=0
    response = requests.get(url,params=params)
    new_row = current

    data=response.json()

    for price in data:
        try:
            if price['deal']['price']['currency'] != 'USD':
                print('Not USD')
                continue
            new_row['date']=price['timestamp']
            default=price['deal']['regular']['amount']
            if default not in df['price']:
                new_row['price']=default
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

            new_row['price']=price['deal']['price']['amount']
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        except:
            continue